In [1]:
import requests
from bs4 import BeautifulSoup
import pyodbc
import re
import time
import random


In [2]:
# Tạo session, tự động giữ cookie
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115 Safari/537.36"
})


In [ ]:
server = "localhost"       
database = "choonline"  
username = "sa"            # user SQL Server
password = "Thao7012"       # mật khẩu

conn = pyodbc.connect(
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server};DATABASE={database};UID={username};PWD={password}"
)
cursor = conn.cursor()
BASE = "https://monngonmoingay.com"

In [ ]:
cursor.execute("""
    SELECT TOP 100 [Tên món ăn], [Link món ăn]
    FROM rawdata_test
    WHERE [Link món ăn] IS NOT NULL
""")

rows = cursor.fetchall()
print(f"✅ Đã lấy {len(rows)} món ăn đầu tiên từ database.")


In [ ]:
for idx, (ten_mon_an, link_mon_an) in enumerate(rows, start=1):
    print(f"\n================= 🍽️ {idx}. {ten_mon_an} =================")

    try:
        # --- Tải HTML ---
        r = session.get(link_mon_an, headers=session.headers, timeout=15)
        if r.status_code != 200:
            print(f"❌ Lỗi tải trang ({r.status_code}), bỏ qua.")
            continue

        soup = BeautifulSoup(r.text, "html.parser")
        print("✅ Tải thành công trang:", ten_mon_an)

        # ========================
        # Tìm UL chứa danh mục
        # ========================
        ul = soup.select_one("div.flex.flex-col.gap-2.items-center > ul")
        if ul:
            danh_muc_list = [li.get_text(strip=True) for li in ul.select("li")]
        else:
            danh_muc_list = []
        print("Danh mục:", danh_muc_list)

        # ========================
        # Tìm Calories (2 trường hợp)
        # ========================
        calories = None

        # --- Trường hợp 1: Dinh dưỡng block ---
        kcal_div = soup.select_one(
            "div.flex.flex-col.gap-2.p-4.bg-\\[var\\(--Gray-04\\)\\].rounded-xl.border.border-\\[var\\(--Primary-03\\)\\]"
        )
        if kcal_div:
            nutrition_block = kcal_div.find("span", string=lambda s: s and "Dinh Dưỡng" in s)
            if nutrition_block:
                strong_tag = nutrition_block.find_next("strong")
                if strong_tag:
                    calories = strong_tag.get_text(strip=True)
                    print("🔥 Calories:", calories)

        # --- Trường hợp 2: Trong bảng ---
        if not calories:
            tables = soup.find_all("table")
            found = False

            for table in tables:
                rows_table = table.find_all("tr")
                for row in rows_table:
                    cells = row.find_all(["td", "th"])
                    for c in cells:
                        text_c = c.get_text(strip=True)
                        numbers_in_parentheses = re.findall(r"\(([^)]*?\d+(?:[\.,]\d+)?[^)]*?)\)", text_c)
                        if numbers_in_parentheses:
                            calories = numbers_in_parentheses
                            found = True
                            break
                    if found:
                        break
                if found:
                    break

        print("Calories tìm được:", calories)

        # ========================
        # Tìm nguyên liệu
        # ========================
        ingredients = []
        ingredients_div = soup.select_one("div.block-nguyenlieu.tab-content#tab-muong ul")
        if ingredients_div:
            for li in ingredients_div.select("li"):
                span = li.find("span")
                if span:
                    ingredients.append(span.get_text(strip=True))
                else:
                    ingredients.append(li.get_text(strip=True))
        print("Nguyên liệu:", ingredients)

        # ========================
        # Tìm Sơ chế
        # ========================
        so_che = []
        so_che_div = soup.select_one("#section-soche")
        if so_che_div:
            for tag in so_che_div.select("p, span, li"):
                text = tag.get_text(strip=True)
                if text:
                    so_che.append(text)
        print("Sơ chế:", so_che)

        # ========================
        # ===== TÌM "CÁCH THỰC HIỆN" (toàn bộ nội dung trong div, trừ thẻ h4) =====
        cach_thuc_hien = []

        section_thuchien = soup.find("div", id="section-thuchien")

        if section_thuchien:
            # Xóa thẻ tiêu đề <h4> để không lấy dòng "Thực Hiện:"
            h4_tag = section_thuchien.find("h4")
            if h4_tag:
                h4_tag.decompose()

            # Lấy toàn bộ văn bản trong các thẻ div/p/li/span con
            for child in section_thuchien.find_all(["div", "p", "li", "span"], recursive=True):
                text = child.get_text(strip=True)
                if text:
                    cach_thuc_hien.append(text)

        print("Cách thực hiện tìm được:", cach_thuc_hien)


        # Tìm Cách dùng
        # ========================
        cach_dung = []
        howtouse_div = soup.select("#section-howtouse > div")
        if howtouse_div:
            cach_dung = [div.get_text(strip=True) for div in howtouse_div]
        print("Cách dùng:", cach_dung)

        # ========================
        # Lưu vào SQL Server
        # ========================
        if (ul and danh_muc_list) or calories or ingredients or so_che or cach_thuc_hien or cach_dung:
            # Nếu calories là list (ví dụ: ['220 kcal/n']) thì nối lại thành chuỗi
            if isinstance(calories, list):
                calories = ", ".join(calories)

            # Giới hạn độ dài tránh lỗi "String or binary data would be truncated"
            if calories and len(calories) > 200:
                calories = calories[:200]

            cursor.execute("""
                UPDATE rawdata_test
                SET [Danh mục món ăn] = ?, 
                    [Calories] = ?, 
                    [Nguyên liệu] = ?, 
                    [Sơ chế] = ?, 
                    [Cách thực hiện] = ?,
                    [Cách dùng] = ?
                WHERE [Link món ăn] = ?
            """, (
                ", ".join(danh_muc_list) if danh_muc_list else None,
                calories,
                ", ".join(ingredients) if ingredients else None,
                " ".join(so_che) if so_che else None,
                "\n".join(cach_thuc_hien) if cach_thuc_hien else None,
                "\n".join(cach_dung) if cach_dung else None,
                link_mon_an
            ))
            conn.commit()
            print("✅ Đã lưu dữ liệu vào SQL.")
        else:
            print("⚠️ Không có dữ liệu để cập nhật.")

        # Nghỉ 2 giây tránh bị chặn
        time.sleep(2)

    except Exception as e:
        print(f"❌ Lỗi khi xử lý {ten_mon_an}: {e}")
        continue

# ========================
# Đóng kết nối
# ========================
conn.close()

In [ ]:

# Lấy các món có cột [Nguyên liệu] bị NULL
# ========================
cursor.execute("""
    SELECT [Tên món ăn], [Link món ăn]
    FROM rawdata_test
    WHERE [Nguyên liệu] IS NULL
""")
rows = cursor.fetchall()
print(f"🔍 Có {len(rows)} món cần cào lại (Nguyên liệu NULL)")

# ========================
# Cấu hình Session
# ========================
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 11.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
})

# ========================
# Vòng lặp cào dữ liệu
# ========================
for idx, (ten_mon_an, link_mon_an) in enumerate(rows, start=1):
    print(f"\n================= 🍽️ {idx}. {ten_mon_an} =================")

    try:
        r = session.get(link_mon_an, headers=session.headers, timeout=15)
        if r.status_code != 200:
            print(f"❌ Lỗi tải trang ({r.status_code}), bỏ qua.")
            continue

        soup = BeautifulSoup(r.text, "html.parser")
        print("✅ Tải thành công trang:", ten_mon_an)

        # ======================== Danh mục ========================
        ul = soup.select_one("div.flex.flex-col.gap-2.items-center > ul")
        danh_muc_list = [li.get_text(strip=True) for li in ul.select("li")] if ul else []
        print("Danh mục:", danh_muc_list)

        # ======================== Calories ========================
        calories = None
        kcal_div = soup.select_one(
            "div.flex.flex-col.gap-2.p-4.bg-\\[var\\(--Gray-04\\)\\].rounded-xl.border.border-\\[var\\(--Primary-03\\)\\]"
        )
        if kcal_div:
            nutrition_block = kcal_div.find("span", string=lambda s: s and "Dinh Dưỡng" in s)
            if nutrition_block:
                strong_tag = nutrition_block.find_next("strong")
                if strong_tag:
                    calories = strong_tag.get_text(strip=True)
                    print("🔥 Calories:", calories)

        if not calories:
            tables = soup.find_all("table")
            for table in tables:
                for row in table.find_all("tr"):
                    for c in row.find_all(["td", "th"]):
                        text_c = c.get_text(strip=True)
                        numbers_in_parentheses = re.findall(r"\(([^)]*?\d+(?:[\.,]\d+)?[^)]*?)\)", text_c)
                        if numbers_in_parentheses:
                            calories = ", ".join(numbers_in_parentheses)
                            break
                    if calories:
                        break
                if calories:
                    break
        print("Calories tìm được:", calories)

        # ======================== Nguyên liệu ========================
        ingredients = []
        ingredients_div = soup.select_one("div.block-nguyenlieu.tab-content#tab-muong ul")
        if ingredients_div:
            for li in ingredients_div.select("li"):
                span = li.find("span")
                ingredients.append(span.get_text(strip=True) if span else li.get_text(strip=True))
        print("Nguyên liệu:", ingredients)

        # ======================== Sơ chế ========================
        so_che = []
        so_che_div = soup.select_one("#section-soche")
        if so_che_div:
            for tag in so_che_div.select("p, span, li"):
                text = tag.get_text(strip=True)
                if text:
                    so_che.append(text)
        print("Sơ chế:", so_che)

        # ======================== Cách thực hiện ========================
        cach_thuc_hien = []
        section_thuchien = soup.find("div", id="section-thuchien")
        if section_thuchien:
            h4_tag = section_thuchien.find("h4")
            if h4_tag:
                h4_tag.decompose()
            for child in section_thuchien.find_all(["div", "p", "li", "span"], recursive=True):
                text = child.get_text(strip=True)
                if text:
                    cach_thuc_hien.append(text)
        print("Cách thực hiện tìm được:", cach_thuc_hien)

        # ======================== Cách dùng ========================
        cach_dung = []
        howtouse_div = soup.select("#section-howtouse > div")
        if howtouse_div:
            cach_dung = [div.get_text(strip=True) for div in howtouse_div]
        print("Cách dùng:", cach_dung)

        # ======================== Cập nhật SQL ========================
        if ingredients or so_che or cach_thuc_hien or cach_dung or calories or danh_muc_list:
            cursor.execute("""
                UPDATE rawdata_test
                SET [Danh mục món ăn] = ?, 
                    [Calories] = ?, 
                    [Nguyên liệu] = ?, 
                    [Sơ chế] = ?, 
                    [Cách thực hiện] = ?,
                    [Cách dùng] = ?
                WHERE [Link món ăn] = ?
            """, (
                ", ".join(danh_muc_list) if danh_muc_list else None,
                calories[:200] if calories and len(calories) > 200 else calories,
                ", ".join(ingredients) if ingredients else None,
                " ".join(so_che) if so_che else None,
                "\n".join(cach_thuc_hien) if cach_thuc_hien else None,
                "\n".join(cach_dung) if cach_dung else None,
                link_mon_an
            ))
            conn.commit()
            print("✅ Đã lưu dữ liệu vào SQL.")
        else:
            print("⚠️ Không có dữ liệu để cập nhật.")

        time.sleep(2)  # tránh bị chặn

    except Exception as e:
        print(f"❌ Lỗi khi xử lý {ten_mon_an}: {e}")
        continue

# ========================
# Đóng kết nối
# ========================
conn.close()
print("\n🎯 Hoàn tất cào lại các món có nguyên liệu NULL.")
